### 実行環境はGoogle Colab

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
from glob import glob
import pandas as pd
import linecache

In [ ]:
datasets = pd.read_csv("/content/drive/MyDrive/catch-up/torch-lstm/datasets.csv")

In [ ]:
datasets.head()

,Unnamed: 0,title,category
0,0,テレビ放送禁止の「お蔵入り恐怖映像」をウェブで公開\n,movie-enter
1,1,ZTE、Intel製「Atom」搭載のAndroidスマートフォン「ZTE Grand X ...,smax
2,2,なぜモテない？！料理男子\n,dokujo-tsushin
3,3,子どもだけにはもったいない！ トイストーリーの世界を体験できそうなカメラ付きミニカー【話題】\n,kaden-channel
4,4,女の本音全開！「彼氏にドン引きした○○な瞬間」\n,dokujo-tsushin


In [ ]:
categories = set(datasets["category"])

In [ ]:
categories

{'dokujo-tsushin',
 'it-life-hack',
 'kaden-channel',
 'livedoor-homme',
 'movie-enter',
 'peachy',
 'smax',
 'sports-watch',
 'topic-news'}

In [ ]:
len(datasets)

7376

In [ ]:
import torch
import torch.nn as nn
embeds = nn.Embedding(10, 6) # (Embedding(単語の合計数, ベクトル次元数))
w1 = torch.tensor([2])
print(embeds(w1))
w2 = torch.tensor([2,4,9])
print(embeds(w2))

tensor([[-1.2028, -0.6900, -1.6108,  0.3982,  1.1026,  1.1454]],
       grad_fn=<EmbeddingBackward>)
tensor([[-1.2028, -0.6900, -1.6108,  0.3982,  1.1026,  1.1454],
        [ 1.2348, -0.4250,  1.8455,  0.9792, -2.2479,  1.6836],
        [ 0.1955, -0.9219, -0.0907, -0.2699,  0.4928,  0.1467]],
       grad_fn=<EmbeddingBackward>)


In [ ]:
!apt install aptitude swig
!aptitude install mecab libmecab-dev mecab-ipadic-utf8 git make curl xz-utils file -y
# 以下で報告があるようにmecab-python3のバージョンを0.996.5にしないとtokezerで落ちる
# https://stackoverflow.com/questions/62860717/huggingface-for-japanese-tokenizer
!pip install mecab-python3==0.996.5
!pip install unidic-lite # これないとMeCab実行時にエラーで落ちる

Reading package lists... Done
Building dependency tree       
Reading state information... Done
aptitude is already the newest version (0.8.10-6ubuntu1).
swig is already the newest version (3.0.12-1).
0 upgraded, 0 newly installed, 0 to remove and 40 not upgraded.
mecab is already installed at the requested version (0.996-5)
libmecab-dev is already installed at the requested version (0.996-5)
mecab-ipadic-utf8 is already installed at the requested version (2.7.0-20070801+main-1)
git is already installed at the requested version (1:2.17.1-1ubuntu0.8)
make is already installed at the requested version (4.1-9.1ubuntu1)
curl is already installed at the requested version (7.58.0-2ubuntu3.14)
xz-utils is already installed at the requested version (5.2.2-1.3)
file is already installed at the requested version (1:5.32-2ubuntu0.4)
mecab is already installed at the requested version (0.996-5)
libmecab-dev is already installed at the requested version (0.996-5)
mecab-ipadic-utf8 is already instal

In [ ]:
import MeCab
import re
import torch

tagger = MeCab.Tagger("-Owakati")

def make_wakati(sentence):
    # MeCabで分かち書き
    sentence = tagger.parse(sentence)
    # 半角全角英数字除去
    sentence = re.sub(r'[0-9０-９a-zA-Zａ-ｚＡ-Ｚ]+', " ", sentence)
    # 記号もろもろ除去
    sentence = re.sub(r'[\．_－―─！＠＃＄％＾＆\-‐|\\＊\“（）＿■×+α※÷⇒—●★☆〇◎◆▼◇△□(：〜～＋=)／*&^%$#@!~`){}［］…\[\]\"\'\”\’:;<>?＜＞〔〕〈〉？、。・,\./『』【】「」→←○《》≪≫\n\u3000]+', "", sentence)
    # スペースで区切って形態素の配列へ
    wakati = sentence.split(" ")
    # 空の要素は削除
    wakati = list(filter(("").__ne__, wakati))
    return wakati

# テスト
test = "【人工知能】は「人間」の仕事を奪った"
print(make_wakati(test))
# ['人工', '知能', 'は', '人間', 'の', '仕事', 'を', '奪っ', 'た']


['人工', '知能', 'は', '人間', 'の', '仕事', 'を', '奪っ', 'た']


In [ ]:
# 単語ID辞書を作成する
word2index = {}
word2index.update({"<pad>":0})
for title in datasets["title"]:
    wakati = make_wakati(title)
    for word in wakati:
        if word in word2index: continue
        word2index[word] = len(word2index)
print("vocab size : ", len(word2index))
print(word2index)

vocab size :  13230
{'<pad>': 0, 'テレビ': 1, '放送': 2, '禁止': 3, 'の': 4, 'お': 5, '蔵': 6, '入り': 7, '恐怖': 8, '映像': 9, 'を': 10, 'ウェブ': 11, 'で': 12, '公開': 13, '製': 14, '搭載': 15, 'スマート': 16, 'フォン': 17, '発表': 18, '欧州': 19, '月': 20, 'に': 21, '発売': 22, 'なぜ': 23, 'モテ': 24, 'ない': 25, '料理': 26, '男子': 27, '子ども': 28, 'だけ': 29, 'は': 30, 'もったいない': 31, 'トイ': 32, 'ストーリー': 33, '世界': 34, '体験': 35, 'でき': 36, 'そう': 37, 'な': 38, 'カメラ': 39, '付き': 40, 'ミニカー': 41, '話題': 42, '女': 43, '本音': 44, '全開': 45, '彼氏': 46, 'ドン': 47, '引き': 48, 'し': 49, 'た': 50, '瞬間': 51, '何かと': 52, 'ネット': 53, 'が': 54, '騒がしかっ': 55, '不審': 56, 'アプリ': 57, '騒動': 58, 'まで': 59, '振り返る': 60, 'フラッシュ': 61, 'バック': 62, 'エクスペンダブルズ': 63, '前代未聞': 64, '予告': 65, '人知れず': 66, '無謀': 67, '戦い': 68, '挑む': 69, '水着': 70, 'グラビア': 71, 'アイドル': 72, '人': 73, 'と': 74, '奇跡': 75, 'スパッツ': 76, '男': 77, '日本': 78, '最果て': 79, '地': 80, '誕生': 81, '映画': 82, 'しん': 83, 'ぼる': 84, 'となり': 85, 'トトロ': 86, '続く': 87, 'プレミア': 88, '上映': 89, '決定': 90, '公式': 91, 'プロバイダサービス': 92, '登場': 93, 'メンバー':

In [ ]:
from sklearn.model_selection import train_test_split
import random
from sklearn.utils import shuffle

In [ ]:
cat2index = {}
for cat in categories:
    if cat in cat2index: continue
    cat2index[cat] = len(cat2index)
print(cat2index)

{'dokujo-tsushin': 0, 'movie-enter': 1, 'peachy': 2, 'topic-news': 3, 'sports-watch': 4, 'livedoor-homme': 5, 'it-life-hack': 6, 'smax': 7, 'kaden-channel': 8}


In [ ]:
def sentence2index(sentence):
    wakati = make_wakati(sentence)
    return [word2index[w] for w in wakati]

def category2index(cat):
    return [cat2index[cat]]

In [ ]:
index_datasets_title_tmp = []
index_datasets_category = []

### 系列の長さの最大値を取得。この長さに他の系列の長さをあわせる

In [ ]:
max_len = 0

In [ ]:
for title, category in zip(datasets["title"], datasets["category"]):
    # print(title)
    # print(category)
    index_title = sentence2index(title)
    index_category = category2index(category)
    # print(index_title)
    # print(index_category)
    # 文字列を数値にindexに変換することは大切なのか...
    index_datasets_title_tmp.append(index_title)
    index_datasets_category.append(index_category)
    if max_len < len(index_title):
        max_len = len(index_title)

### 系列の長さを揃えるために短い系列にパディングを追加
### 後ろパディングだと正しく学習できなかったので、前パディング

In [ ]:
index_datasets_title = []
for title in index_datasets_title_tmp:
  for i in range(max_len - len(title)):
    title.insert(0, 0) # 前パディング
#     title.append(0)　# 後ろパディング
  index_datasets_title.append(title)
print(index_datasets_title)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
train_x, test_x, train_y, test_y = train_test_split(index_datasets_title, index_datasets_category, train_size=0.7)

In [ ]:
# データをバッチでまとめるための関数
def train2batch(title, category, batch_size=100):
  title_batch = []
  category_batch = []
  title_shuffle, category_shuffle = shuffle(title, category)
  for i in range(0, len(title), batch_size):
    title_batch.append(title_shuffle[i:i+batch_size])
    category_batch.append(category_shuffle[i:i+batch_size])
  return title_batch, category_batch

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 15558448342919519042, name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 11345264640
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 17116504340558657837
 physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:04.0, compute capability: 3.7"]

In [ ]:
class LSTMClassifier(nn.Module):
    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size):
        super(LSTMClassifier, self).__init__()
        self.hidden_dim = hidden_dim
        # <pad>の単語IDが0なので、padding_idx=0としている
        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        # batch_first=Trueが大事！
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        self.hidden2tag = nn.Linear(hidden_dim, tagset_size)
        self.softmax = nn.LogSoftmax()

    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
        #embeds.size() = (batch_size × len(sentence) × embedding_dim)
        _, lstm_out = self.lstm(embeds)
        # lstm_out[0].size() = (1 × batch_size × hidden_dim)
        tag_space = self.hidden2tag(lstm_out[0])
        # tag_space.size() = (1 × batch_size × tagset_size)

        # (batch_size × tagset_size)にするためにsqueeze()する
        tag_scores = self.softmax(tag_space.squeeze())
        # tag_scores.size() = (batch_size × tagset_size)

        return tag_scores

In [ ]:
EMBEDDING_DIM = 200
HIDDEN_DIM = 128
VOCAB_SIZE = len(word2index)
TAG_SIZE = len(categories)
# to(device)でモデルがGPU対応する
model = LSTMClassifier(EMBEDDING_DIM, HIDDEN_DIM, VOCAB_SIZE, TAG_SIZE).to(device)
loss_function = nn.NLLLoss()
# SGDからAdamに変更。特に意味はなし
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
losses = []
for epoch in range(100):
    all_loss = 0
    title_batch, category_batch = train2batch(train_x, train_y)
    for i in range(len(title_batch)):
        batch_loss = 0

        model.zero_grad()

        # 順伝搬させるtensorはGPUで処理させるためdevice=にGPUをセット
        title_tensor = torch.tensor(title_batch[i], device=device)
        # category_tensor.size() = (batch_size × 1)なので、squeeze()
        category_tensor = torch.tensor(category_batch[i], device=device).squeeze()

        out = model(title_tensor)

        batch_loss = loss_function(out, category_tensor)
        batch_loss.backward()
        optimizer.step()

        all_loss += batch_loss.item()
    print("epoch", epoch, "\t" , "loss", all_loss)
    if all_loss < 0.1: break
print("done.")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


epoch 0 	 loss 235.57591086626053
epoch 1 	 loss 68.05441150069237
epoch 2 	 loss 17.04076342098415
epoch 3 	 loss 4.65863793855533
epoch 4 	 loss 2.0366284963674843
epoch 5 	 loss 1.3301582244457677
epoch 6 	 loss 1.0574431918212213
epoch 7 	 loss 0.8920795148587786
epoch 8 	 loss 0.8171967128873803
epoch 9 	 loss 0.7604854549572337
epoch 10 	 loss 0.7387701683910564
epoch 11 	 loss 0.6920981075672898
epoch 12 	 loss 0.6838753410265781
epoch 13 	 loss 0.6577136726555182
epoch 14 	 loss 0.6629296440369217
epoch 15 	 loss 0.6645617022077204
epoch 16 	 loss 0.6283206385342055
epoch 17 	 loss 0.6118035173567478
epoch 18 	 loss 0.6151796687336173
epoch 19 	 loss 0.6010671682743123
epoch 20 	 loss 0.6037157931277761
epoch 21 	 loss 0.6027713052244508
epoch 22 	 loss 0.6095133743547194
epoch 23 	 loss 0.5952725926981657
epoch 24 	 loss 0.5862193660013872
epoch 25 	 loss 0.5908712245927745
epoch 26 	 loss 0.606775183319769
epoch 27 	 loss 0.5810129634519399
epoch 28 	 loss 0.580307146316045
e

In [ ]:
test_num = len(test_x)
a = 0
with torch.no_grad():
    title_batch, category_batch = train2batch(test_x, test_y)

    for i in range(len(title_batch)):
        title_tensor = torch.tensor(title_batch[i], device=device)
        category_tensor = torch.tensor(category_batch[i], device=device)

        out = model(title_tensor)
        _, predicts = torch.max(out, 1)
        for j, ans in enumerate(category_tensor):
            if predicts[j].item() == ans.item():
                a += 1
print("predict : ", a / test_num)
# めちゃ精度高くない？？？
# predict :  0.6967916854948034

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


predict :  0.990736556710348
